## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hay River,CA,60.8156,-115.7999,10.38,85,90,16.11,snow
1,1,Puerto Ayora,EC,-0.7393,-90.3518,76.96,86,50,8.99,scattered clouds
2,2,Revda,RU,56.8010,59.9303,23.81,85,100,12.73,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,54.23,90,75,8.05,broken clouds
4,4,Kavieng,PG,-2.5744,150.7967,81.61,75,100,1.90,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,76.96,86,50,8.99,scattered clouds
4,4,Kavieng,PG,-2.5744,150.7967,81.61,75,100,1.90,light rain
6,6,Ribeira Grande,PT,38.5167,-28.7000,64.80,82,40,17.27,scattered clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,81.81,78,55,19.46,broken clouds
9,9,Kyaukse,MM,21.6000,96.1333,69.21,92,70,2.68,broken clouds
10,10,Piacabucu,BR,-10.4056,-36.4344,79.18,77,28,13.96,scattered clouds
11,11,Rikitea,PF,-23.1203,-134.9692,73.49,65,4,7.67,clear sky
12,12,Bengkulu,ID,-3.8004,102.2655,75.11,88,100,4.29,overcast clouds
13,13,Fallon,US,39.4735,-118.7774,65.43,37,75,9.22,broken clouds
15,15,Huarmey,PE,-10.0681,-78.1522,67.87,79,99,8.77,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                353
City                   353
Country                351
Lat                    353
Lng                    353
Max Temp               353
Humidity               353
Cloudiness             353
Wind Speed             353
Current Description    353
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                351
City                   351
Country                351
Lat                    351
Lng                    351
Max Temp               351
Humidity               351
Cloudiness             351
Wind Speed             351
Current Description    351
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,
4,Kavieng,PG,81.61,light rain,-2.5744,150.7967,
6,Ribeira Grande,PT,64.80,scattered clouds,38.5167,-28.7000,
8,Hithadhoo,MV,81.81,broken clouds,-0.6000,73.0833,
9,Kyaukse,MM,69.21,broken clouds,21.6000,96.1333,
10,Piacabucu,BR,79.18,scattered clouds,-10.4056,-36.4344,
11,Rikitea,PF,73.49,clear sky,-23.1203,-134.9692,
12,Bengkulu,ID,75.11,overcast clouds,-3.8004,102.2655,
13,Fallon,US,65.43,broken clouds,39.4735,-118.7774,
15,Huarmey,PE,67.87,overcast clouds,-10.0681,-78.1522,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city. 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
4,Kavieng,PG,81.61,light rain,-2.5744,150.7967,Nusa Island Retreat
6,Ribeira Grande,PT,64.80,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
8,Hithadhoo,MV,81.81,broken clouds,-0.6000,73.0833,Scoop Guest House
9,Kyaukse,MM,69.21,broken clouds,21.6000,96.1333,Shwe Thit Sar Guesthouse
...,...,...,...,...,...,...,...
673,Laranjeiras Do Sul,BR,64.27,clear sky,-25.4078,-52.4161,Hotel Palmeiras
675,Nanyamba,TZ,75.52,few clouds,-10.6833,39.8333,Busati Lodge Nanyamba
676,Salta,AR,79.09,broken clouds,-24.7859,-65.4117,Hotel Alejandro I
677,Cap Malheureux,MU,75.69,few clouds,-19.9842,57.6142,Pereybere Hotel & Spa


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} °F</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))